In [2]:
#transformations

mx=0b100011011

#text->4x4 matrix
#in: text; out: state
def text2matrix(text):
    state= [[0,0,0,0],
            [0,0,0,0],
            [0,0,0,0],
            [0,0,0,0]]
    index =0
    # text = hex(int(text))
    for column in range(4):
        for row in range(4):
            state[row][column]=(int(text[index:index+2], base=16))
            index+=2
    
    return state

#text->4x4 matrix inv
#in: text; out: state
def invtext2matrix(text):
    state= [[0,0,0,0],
            [0,0,0,0],
            [0,0,0,0],
            [0,0,0,0]]
    index =0
    # text = hex(int(text))
    for column in range(4):
        for row in range(4):
            state[row][column]=(text[index:index+2])
            index+=2
    
    return state

#4x4 matrix -> plaintext
#in: state; out: text
def matrix2text(matrix):
    text =''
    for column in range(4):
        for row in range(4):
            text+=matrix[row][column]
    return text


#SubBytes
#in: state; out: state
#This code is borrowed from the last project, in which we implemented the S-box and GF(2^8) arithmetic
#multiplication
def multiply(a, b):
    result = 0
    while a != 0:
        if a & 1 == 1:
            result ^= b
        a >>= 1
        b <<= 1
    return result

#the inverse of the number
def inverse(number):
    if number == 0:
        return 0
    for i in range(1, 256):
        if helper(number, i) == 1:
            return i

#creates the sbox
def create_sbox():
    for row in range(16):
        for column in range(16):
            x = column + row * 16 
            value = format(sbox(x), "02x")
            # print(value, end=' ')
        # print()
        
#creates the inverse sbox
def create_inverse_sbox():
    for row in range(16):
        for column in reversed(range(16)):
            x = column + row * 16 
            value = format(inverse_sbox(x), "02x")
            # print(value, end=' ')
        # print()
        
#helper functions

#looking for the degree of the #
def bits_count(byte):
    number =0
    while byte!=0:
        number+=1
        byte>>=1
    return number

#applies the irreducible polynomial
def mod(a, mod):
    # print(a, mod)
    bits_mod = bits_count(mod)
    while True:
        bits_a = bits_count(a)
        if bits_mod > bits_a:
            return a
        shift = mod << (bits_a - bits_mod)
        a ^= shift

#unifies the multiplication 
def helper(x, y):
    value = multiply(x, y)
    mx = 0b100011011
    return mod(value, mx)

#applies the affine transformation to the number
def aff_trans(matrix, inverse, vector):
    result = 0
    for i in reversed(range(8)):
        row = (matrix >> 8 * i) & 0xff 
        bit = dot_product(row, inverse)
        result ^= (bit << i)
    return result ^ vector

#applies the affine transformation to the number
def aff_trans_reverse(matrix, number, vector):
    result = 0
    for i in reversed(range(8)):
        row = (matrix >> 8 * i) & 0xff 
        bit = dot_product(row, number)
        result ^= (bit << i)
    return result ^ vector


#part of the affine transormation
def dot_product(a, b):
    product = 0  
    result = a & b

    while result != 0:
        product ^= result & 1
        result >>= 1
    return product 

#joins the matrix, vector and inverse to do the affine transformation
#input is a 
def sbox(x):
    matrix = 0b1111100001111100001111100001111110001111110001111110001111110001
    vector = 0b01100011
    inverse_number = inverse(x)

    return aff_trans(matrix, inverse_number, vector)


#InvSubBytes
#in: state; out: state
def inverse_sbox(x):
    matrix = 0b0101001000101001100101000100101000100101100100100100100110100100
    vector = 0b10100000
    affine_value = aff_trans_reverse(matrix, x, vector)
    inverse_number = inverse(affine_value)
    
    return inverse_number


#ShiftRows
#in: state; out: state
def shiftRows(state):
    for row in range(4):
        if row ==0:
            continue
        elif row ==1:
            state[row] = shiftLeft(state[row])
        elif row ==2:
            state[row] = shiftTwice(state[row])
        else:
            state[row] = shiftRight(state[row])
    return state

def shiftLeft(row):
 
    # In n<<d, last d bits are 0.
    # To put first 3 bits of n at
    # last, do bitwise or of n<<d
    # with n >>(INT_BITS - d)
    # print(row)
    # return (row << 8)|(row >> (32 - 8)) 
    # print(row)
    currentValue = row[3]
    for index in reversed(range(4)):
        moddedIndex = (index-1) % 4
        prevValue = row[moddedIndex]
        row[moddedIndex] = currentValue
        currentValue = prevValue
        
    return row
        
    
# rotate n by d bits
def shiftRight(row):
 
    # In n>>d, first d bits are 0.
    # To put last 3 bits of at
    # first, do bitwise or of n>>d
    # with n <<(INT_BITS - d)
    # return (n >> 8)|(n << (32 - 8)) & 0xFFFFFFFF
    currentValue = row[0]
    for index in range(4):
        moddedIndex = (index+1) % 4
        nextValue = row[moddedIndex]
        row[moddedIndex] = currentValue
        currentValue = nextValue
    return row


def shiftTwice(row):
 
    # In n>>d, first d bits are 0.
    # To put last 3 bits of at
    # first, do bitwise or of n>>d
    # with n <<(INT_BITS - d)
    # return (n >> 16)|(n << (32 - 16)) & 0xFFFFFFFF
    
    currentValue = row[0]
    for index in [0,2]:
        moddedIndex = (index +2) %4
        nextValue = row[moddedIndex]
        row[moddedIndex] = currentValue
        currentValue = nextValue
        
    currentValue = row[1]
    for index in [1,3]:
        moddedIndex = (index +2) %4
        nextValue = row[moddedIndex]
        row[moddedIndex] = currentValue
        currentValue = nextValue
    
    return row

#InvShiftRows
#in: state; out: state
def invShiftRows(state):
    for row in range(4):
        if row ==0:
            continue
        elif row ==1:
            state[row] = shiftRight(state[row])
        elif row ==2:
            state[row] = shiftTwice(state[row])
        else:
            state[row] = shiftLeft(state[row])
    return state

#MixColumns
#in: state; out: state

def matrixMultiplication(column):
    temp = []
    # print(column)
    for index in range(4):
        if index==0:
            b0p = add(add(add(helper(column[0],2), helper(column[1], 3)), helper(column[2],1)), 
                      helper(column[3],1))
            temp.append(b0p)
        elif index==1:
            b1p = add(add(add(helper(column[0],1), helper(column[1], 2)), helper(column[2],3)), 
                      helper(column[3],1))
            temp.append(b1p)
        elif index==2:
            b2p = add(add(add(helper(column[0],1), helper(column[1], 1)), helper(column[2],2)), 
                      helper(column[3],3))
            temp.append(b2p)
        else:
            b3p = add(add(add(helper(column[0],3), helper(column[1], 1)), helper(column[2],1)), 
                      helper(column[3],2))
            temp.append(b3p)
    # print('tempt:  ', temp)
    return temp

def mixColumns(state):
    transposedState = []

    #helper is the multiplication
    for column in range(len(state[0])):
        newColumn = matrixMultiplication([row[column] for row in state])
        transposedState.append(newColumn)
    for column in range(4):
        for row in range(4):
            state[row][column] = transposedState[column][row]

    return state



#InvMixColumns
#in: state; out

def invmatrixMultiplication(column):
    temp = []
    # print(column)
    for index in range(4):
        if index==0:
            b0p = add(add(add(helper(column[0],14), helper(column[1], 11)), helper(column[2],13)), 
                      helper(column[3],9))
            temp.append(b0p)
        elif index==1:
            b1p = add(add(add(helper(column[0],9), helper(column[1], 14)), helper(column[2],11)), 
                      helper(column[3],13))
            temp.append(b1p)
        elif index==2:
            b2p = add(add(add(helper(column[0],13), helper(column[1], 9)), helper(column[2],14)), 
                      helper(column[3],11))
            temp.append(b2p)
        else:
            b3p = add(add(add(helper(column[0],11), helper(column[1], 13)), helper(column[2],9)), 
                      helper(column[3],14))
            temp.append(b3p)
    # print('tempt:  ', temp)
    return temp


def invMixColumns(state):
    transposedState = []

    #helper is the multiplication
    for column in range(len(state[0])):
        newColumn = invmatrixMultiplication([row[column] for row in state])
        transposedState.append(newColumn)
    for column in range(4):
        for row in range(4):
            state[row][column] = transposedState[column][row]

    return state

#AddRoundKey
#in: state, expansion key; out: state

def addRoundKey(state, key):
    transposedState = []
    for column in range(4):
        newColumn = addColumns([row[column] for row in state], [row[column] for row in key])
        transposedState.append(newColumn)
    # print('transposed', transposedState)
    for column in range(4):
        for row in range(4):
            state[row][column] = transposedState[column][row]
    return state

def addColumns(stateColumn, keyColumn):
    column=[]
    # print(stateColumn)
    # print()
    # print('statecolumn: ', stateColumn, ' keyColumn: ', keyColumn)
    for x in range(4):
        # print(keyColumn)
        column.append(add(int(stateColumn[x]), int(keyColumn[x], 16)))
    return column

#additon
def add(a,b):
    #efficient addition, simple bit-wise XOR
    # print(a, b)
    if a ==0:
        return b
    
    if b ==0:
        return a
    
    
    
#     while b != 0:
#         stored = a&b 
        
#         a = a^b 
        
#         b = stored<<1 
    
    return a^b
    # return a % mx

                      
                      
#KeyExpansion
#in: key; out: array of expanded key
def keyExpansion(key):
    matrix2return =key
    temp = matrix2return
    roundMatrix = []
    w0=''
    w1=''
    w2=''
    w3=''
    for column in range(4):
            #compressing the column into a word
            roundMatrix.append([row[column] for row in key])
            # print('initial round matrix: ' , roundMatrix)
            
    for element in range(4):
        w0+=roundMatrix[0][element]
        w1+=roundMatrix[1][element]
        w2+=roundMatrix[2][element]
        w3+=roundMatrix[3][element]

    roundMatrix = [[w0,w1,w2,w3]]


            
    # print('stored initial matrix')
    for i in range(1, 11):
        #roundColumn will hold the 4 words for the round, they will get transferred to the 
        #roundMatrix, which holds all of the rounds. 
        roundColumn = []
        # print('round', i)
        # print('round: ', i, roundMatrix)
        
        #this loop is going thorugh the individual elements of the word, 
        for column in range(4):
            # print('xolumn: ', column) 
            # print('round: column: ', i, column)
            if column %4 ==0:
                #g function 
                helper=[]
                index=0
                while index < 8:
                    helper.append(roundMatrix[i-1][3][index: index+2])
                    index+=2
                # print(helper)
                
                #shiftedword is an array representation of the w3
                shiftedWord = shiftLeft(helper)
                
                # print('shiftedWord: ', shiftedWord)
                
                subbedWord = ''
                roundNumber = 2**i
                for element in shiftedWord:
                    # print(element)
                    subbedByte = sbox(int(element, 16))
                    subbedWord+=(hex(subbedByte)[2:])
                
                
                # print('subbedWord: ', subbedWord)
                
                #xoriing the whole word with the round number    
                rotatedword = add (roundNumber, int(subbedWord, 16))
                # print('rotatedword: ', rotatedword)
                roundColumn.append(hex(add(rotatedword, int(roundMatrix[i-1][column], 16)))[2:])
                
            # print('matrix: roundColumn: ',roundMatrix[i-1][column], roundColumn[column-1])

            if column % 4 !=0:
                roundColumn.append(hex(add(int(roundMatrix[i-1][column], 16), 
                                           int(roundColumn[column-1], 16)))[2:])
            # print('roundColumn: ', roundColumn)
        
        roundMatrix.append(roundColumn)
        # print('before loop: ', roundMatrix)
#         for column in range(4):
#             for row in range(4):
#                 temp[row][column] = roundMatrix[column][row]
        
#         matrix2return.append(temp)
    
    # print('temp: ', temp)
    
    # print()
    # print('roundMatrix: ', roundMatrix)
    return roundMatrix
            

#encryption
#in: plaintext, key; out: ciphertext

def AES_encryption(plaintext, key):
    
    #calling for the expansion of the key
    expandedKey = keyExpansion(key)
    
    # for x in expandedKey:
    #     print(x)
    # print(expandedKey)
    
    #from text to matrix
    state = text2matrix(plaintext)
    
    #initializing the initial state
    state=addRoundKey(state, expandedKey)
    
    for i in range(1, 10):
        #call subkey
        for row in range(4):
            for column in range(4):
                state[row][column] = sbox(state[row][column])
                
        #call shift row
        state = shiftRows(state) 
        if(i<9):
            #call mix columns
            state = mixColumns(state)
            # print(state)
        #call add round key
        state = addRoundKey(state, expandedKey[i])
        # print(state)
    for row in range(4):
        for column in range(4):
            state[row][column] = str(state[row][column])[2:]
            
            
    ciphertext = matrix2text(state)

    return ciphertext

#decryption
#in: ciphertext, key; out: plaintext

def AES_decryption(ciphertext, key):
    
    #calling for the expansion of the key
    expandedKey = keyExpansion(key)
    
    # for x in expandedKey:
    #     print(x)
    # print(expandedKey)
    
    #from text to matrix
    state = invtext2matrix(ciphertext)
    
    #initializing the initial state
    # print(expandedKey)
    state=addRoundKey(state, expandedKey[10:])
    
    for i in reversed(range(1, 10)):
        #call shift row
        state = invShiftRows(state) 
        
        #call subkey
        for row in range(4):
            for column in range(4):
                state[row][column] = inverse_sbox(state[row][column])
       
        #call add round key
        state = addRoundKey(state, expandedKey[i])
        
        if(i<9):
            #call mix columns
            state = InvMixColumns(state)
        
    
    plaintext = matrix2text(state)
    return plaintext



In [ ]:
# print(chr(171))
w0='0f157169'
w1='47d9e859'
w2='0cb7add6'
w3='af7f6798'
key = w0+w1+w2+w3

plaintext='0189fe7621abdc5445cdba3267ef9810'
print(matrix2text(text2matrix(plaintext)))


In [3]:
key = [['0f', '47', '0c', 'af'],
       ['15', 'd9', 'b7', '7f'],
       ['71','e8','ad','67'],
       ['c9','59','d6','98']]

plaintext='0189fe7623abdc5445cdba3267ef9810'
# print(len(plaintext))
plaintext='0189fe7623abdc5445cdba3267ef9810'


# state = text2matrix(plaintext)
# print(state)
# lol = shiftRows(state)
# print(lol)
# testKey = [[2, 3,1,1],
#             [1,2,3,1],
#             [1,1,2,3],
#             [3,1,1,2]]
# print(addRoundKey(state, testKey))

# print(mixColumns(state))


# AES_decyption(AES_encryption(plaintext,key), key)
ciphertext= AES_encryption(plaintext,key)
print(ciphertext)

# ciphertext+='000'
ciphertext = '0123456789abcdeffedcba9876543210'
# print(len(ciphertext))

# cipherMatrix = invtext2matrix(ciphertext)

AES_decryption(ciphertext, key)

9429110


IndexError: list index out of range